In [1]:
import sympy
from sympy import I, conjugate, pi, oo
from sympy.functions import exp
from sympy.matrices import Identity, MatrixSymbol
import numpy as np

# \#3

## Fourier matrix with $\omega$

In [2]:
ω = sympy.Symbol("omega")

def omegaPow(idx):
    return ω**(idx) 

def fourierGenerator(N):
    return sympy.Matrix(N, N, lambda m, n: omegaPow(m * n))

assert fourierGenerator(2) == sympy.Matrix([[1, 1], [1, ω]])

fourierGenerator(4)

Matrix([
[1,        1,        1,        1],
[1,    omega, omega**2, omega**3],
[1, omega**2, omega**4, omega**6],
[1, omega**3, omega**6, omega**9]])

In [3]:
fourierGenerator(4)

Matrix([
[1,        1,        1,        1],
[1,    omega, omega**2, omega**3],
[1, omega**2, omega**4, omega**6],
[1, omega**3, omega**6, omega**9]])

## Complex conjugate F

In [4]:
conjugate(fourierGenerator(4))

Matrix([
[1,                   1,                   1,                   1],
[1,    conjugate(omega), conjugate(omega)**2, conjugate(omega)**3],
[1, conjugate(omega)**2, conjugate(omega)**4, conjugate(omega)**6],
[1, conjugate(omega)**3, conjugate(omega)**6, conjugate(omega)**9]])

## Substitute $\omega=e^\frac{i2{\pi}mn}{N}$

In [5]:
def omegaSubstirution(idx, N):
    return sympy.exp((I * 2 * pi * idx) / N)


assert omegaSubstirution(1, 4) == I
assert omegaSubstirution(8, 8) == 1
assert omegaSubstirution(3, 6) == -1

## Generate Fourier matrix with values

In [6]:
def fourierGeneratorWithExp(N):
    return sympy.Matrix(N, N, lambda m, n: omegaSubstirution(m * n, N))


assert fourierGeneratorWithExp(2) == sympy.Matrix([[1, 1], [1, -1]])

F4 = fourierGeneratorWithExp(4)

F4

Matrix([
[1,  1,  1,  1],
[1,  I, -1, -I],
[1, -1,  1, -1],
[1, -I, -1,  I]])

## Matrix conjugate

In [7]:
F4Conj = conjugate(F4)

assert Identity(4).as_explicit() == (1 / 4) * F4 * F4Conj

F4Conj

Matrix([
[1,  1,  1,  1],
[1, -I, -1,  I],
[1, -1,  1, -1],
[1,  I, -1, -I]])

## Conjugate generator with $\omega$

In [8]:
def fourierConjGenerator(N):
    return sympy.Matrix(N, N, lambda m, n: omegaPow(0 if m == 0 else (N - m) * n))


fourierConjGenerator(4)

Matrix([
[1,        1,        1,        1],
[1, omega**3, omega**6, omega**9],
[1, omega**2, omega**4, omega**6],
[1,    omega, omega**2, omega**3]])

## Conjugate generator with values

In [9]:
def fourierConjGeneratorWithExp(N):
    return sympy.Matrix(
        N, N, lambda m, n: omegaSubstirution(0 if m == 0 else (N - m) * n, N))


F4ConjWithExp = fourierConjGeneratorWithExp(4)

assert F4Conj == F4ConjWithExp

F4ConjWithExp

Matrix([
[1,  1,  1,  1],
[1, -I, -1,  I],
[1, -1,  1, -1],
[1,  I, -1, -I]])

## Permutation Generator

In [10]:
def generatePermutationMatrix(N):
    return np.vstack((np.hstack((np.array([1]), np.zeros(
        N - 1, dtype=int))), np.zeros((N - 1, N), dtype=int))) + np.fliplr(
            np.diagflat(np.ones(N - 1, dtype=int), -1))


assert np.all(
    generatePermutationMatrix(4) == np.array([[1, 0, 0, 0], [0, 0, 0, 1],
                                              [0, 0, 1, 0], [0, 1, 0, 0]]))

generatePermutationMatrix(4)

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

## $$F=P{\cdot}\overline{F}$$

In [11]:
P4 = generatePermutationMatrix(4)

assert F4 == P4 * F4Conj

P4 * F4Conj

Matrix([
[1,  1,  1,  1],
[1,  I, -1, -I],
[1, -1,  1, -1],
[1, -I, -1,  I]])

## $$P^2 = I$$

In [12]:
assert np.all(np.linalg.matrix_power(P4, 2) == np.identity(4, dtype=int))

## $$\frac{1}{N}{\cdot}F^2 = P$$

In [13]:
assert np.all(((1 / 4) * np.linalg.matrix_power(F4, 2)).astype(int) == P4)

# \#4

In [14]:
ID = sympy.Matrix(
    np.hstack((np.vstack((np.identity(2, dtype=int), np.identity(2,
                                                                 dtype=int))),
               np.vstack((np.identity(2, dtype=int) * np.diag(F4)[0:2],
                          -np.identity(2, dtype=int) * np.diag(F4)[0:2])))))

ID

Matrix([
[1, 0,  1,  0],
[0, 1,  0,  I],
[1, 0, -1,  0],
[0, 1,  0, -I]])

In [15]:
ID.inv()

Matrix([
[1/2,    0,  1/2,   0],
[  0,  1/2,    0, 1/2],
[1/2,    0, -1/2,   0],
[  0, -I/2,    0, I/2]])

In [16]:
F2 = fourierGeneratorWithExp(2)
Zeros2 = np.zeros((2, 2), dtype=int)

F22 = sympy.Matrix(
    np.array(np.vstack((np.hstack((F2, Zeros2)), np.hstack((Zeros2, F2))))))

F22

Matrix([
[1,  1, 0,  0],
[1, -1, 0,  0],
[0,  0, 1,  1],
[0,  0, 1, -1]])

In [17]:
F22.inv()

Matrix([
[1/2,  1/2,   0,    0],
[1/2, -1/2,   0,    0],
[  0,    0, 1/2,  1/2],
[  0,    0, 1/2, -1/2]])

In [18]:
EvenOddP = sympy.Matrix([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0],
                         [0, 0, 0, 1]])
EvenOddP

Matrix([
[1, 0, 0, 0],
[0, 0, 1, 0],
[0, 1, 0, 0],
[0, 0, 0, 1]])

In [19]:
assert ID * F22 * EvenOddP == F4

ID * F22 * EvenOddP

Matrix([
[1,  1,  1,  1],
[1,  I, -1, -I],
[1, -1,  1, -1],
[1, -I, -1,  I]])

In [20]:
F4

Matrix([
[1,  1,  1,  1],
[1,  I, -1, -I],
[1, -1,  1, -1],
[1, -I, -1,  I]])

# \#5

$$
 F^{T}_{N} = 
\left(
    \begin{bmatrix}
       I_{N/2} & D_{N/2}  \\[0.3em]
       I_{N/2} & -D_{N/2} \\[0.3em]
     \end{bmatrix}
     \cdot
     \begin{bmatrix}
       F_{N/2} &   \\[0.3em]
        & F_{N/2} \\[0.3em]
     \end{bmatrix}
     \cdot
     \begin{bmatrix}
       even_{N/2} & even_{N/2}  \\[0.3em]
       odd_{N/2} & odd_{N/2} \\[0.3em]
     \end{bmatrix}
 \right)^T
     =
    \begin{bmatrix}
       even_{N/2} & odd_{N/2}  \\[0.3em]
       even_{N/2} & odd_{N/2} \\[0.3em]
     \end{bmatrix}
    \cdot
    \begin{bmatrix}
        F_{N/2} &   \\[0.3em]
        & F_{N/2} \\[0.3em]
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
        I_{N/2} & I_{N/2} \\[0.3em]
        D_{N/2} & -D_{N/2} \\[0.3em]
    \end{bmatrix}
 $$

In [21]:
EvenOddP * F22 * sympy.Matrix.transpose(ID)

Matrix([
[1,  1,  1,  1],
[1,  I, -1, -I],
[1, -1,  1, -1],
[1, -I, -1,  I]])

In [22]:
EvenOddP * F22 * sympy.Matrix.transpose(ID) == F4

True

# \#6

### Based on Permutation Matrix for the Conjugate Matrix - even / odd Permutation

In [23]:
N = 6

P = generatePermutationMatrix(N)

EvenOddP = sympy.Matrix(
    np.vstack((
        P[0],
        P[np.arange(N-2, 1, -2)],
        P[np.arange(N-1, 0, -2)]
    ))
)

EvenOddP

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 1]])

In [24]:
def generatePermutationFourierMatrix(N):
    P = generatePermutationMatrix(N)
    return sympy.Matrix(
        np.vstack((
            P[0],
            P[np.arange(N-2, 1, -2)],
            P[np.arange(N-1, 0, -2)]
        ))
    )

assert np.all(generatePermutationFourierMatrix(4) == sympy.Matrix([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
]))

### Shape of the Permutation Matrix
$$
\begin{bmatrix}
    even_{N/2} & even_{N/2}  \\[0.3em]
    odd_{N/2} & odd_{N/2} \\[0.3em]
\end{bmatrix}
$$

### N = 8 Fourier Permutation Matrix

In [25]:
generatePermutationFourierMatrix(8)

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])

In [26]:
generatePermutationFourierMatrix(6)

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 1]])

## Generate Fourier Matrices

### Generate Fourier Blocks for basic Matrix:

#### Half Eye: $I_{N/2}$
#### Half Fourier Diagonal: $D_{N/2}$
#### Half Fourier: $F_{N/2}$

In [27]:
def generateFourierBlocks(N, fourierGenerator):
    half = int(N/2)
    quarterEye = sympy.Matrix.eye(half, half)
    quarterZeroes = sympy.Matrix.zeros(half, half)
    halfDiagFourier = sympy.Matrix.diag(
        np.diagonal(fourierGenerator(N))
    )[:half, :half]
    halfFourier = fourierGenerator(half)
    return (quarterEye, quarterZeroes, halfDiagFourier, halfFourier)

In [28]:
Blocks4 = generateFourierBlocks(4, fourierGenerator)

assert Blocks4 == (
    sympy.Matrix([
        [1, 0],
        [0, 1]
    ]),
    sympy.Matrix([
        [0, 0],
        [0, 0]
    ]),
    sympy.Matrix([
        [1, 0],
        [0, ω]
    ]),
    sympy.Matrix([
        [1, 1],
        [1, ω]
    ])
)

In [29]:
Blocks4Complex = generateFourierBlocks(4, fourierGeneratorWithExp)

assert Blocks4Complex == (
    sympy.Matrix([
        [1, 0],
        [0, 1]
    ]),
    sympy.Matrix([
        [0, 0],
        [0, 0]
    ]),
    sympy.Matrix([
        [1, 0],
        [0, I]
    ]),
    sympy.Matrix([
        [1, 1],
        [1, -1]
    ])
)

In [30]:
def composeFourierMatricesFromBlocks(N, quarterEye, quarterZeroes, halfDiagFourier, halfFourier):
    return (
        sympy.Matrix.vstack(
            sympy.Matrix.hstack(
                quarterEye,
                halfDiagFourier
            ),
            sympy.Matrix.hstack(
                quarterEye,
                -halfDiagFourier
            )
        ),
        sympy.Matrix.vstack(
            sympy.Matrix.hstack(
                halfFourier,
                quarterZeroes
            ),
            sympy.Matrix.hstack(
                quarterZeroes,
                halfFourier
            )
        ),
        generatePermutationFourierMatrix(N)
    )

In [31]:
def createFourierMatrices(N, fourierGenerator):
    (quarterEye, quarterZeroes, halfDiagFourier, halfFourier) = generateFourierBlocks(N, fourierGenerator)
    return composeFourierMatricesFromBlocks(N, quarterEye, quarterZeroes, halfDiagFourier, halfFourier)

## Generate Fourier Matrices 

### IdentityAndDiagonal: $
\begin{bmatrix}
   I_{N/2} & D_{N/2}  \\[0.3em]
   I_{N/2} & -D_{N/2} \\[0.3em]
 \end{bmatrix}
$
 
### FourierHalfNSize: $
\begin{bmatrix}
   F_{N/2} &   \\[0.3em]
    & F_{N/2} \\[0.3em]
 \end{bmatrix}
$

### EvenOddPermutation: $
\begin{bmatrix}
    even_{N/2} & even_{N/2}  \\[0.3em]
    odd_{N/2} & odd_{N/2} \\[0.3em]
\end{bmatrix}
$

### Full picture
$$
F_{N}=\begin{bmatrix}
   I_{N/2} & D_{N/2}  \\[0.3em]
   I_{N/2} & -D_{N/2} \\[0.3em]
 \end{bmatrix}
 \cdot
 \begin{bmatrix}
   F_{N/2} &   \\[0.3em]
    & F_{N/2} \\[0.3em]
 \end{bmatrix}
 \cdot
 \begin{bmatrix}
   even_{N/2} & even_{N/2}  \\[0.3em]
   odd_{N/2} & odd_{N/2} \\[0.3em]
 \end{bmatrix}
 $$

In [32]:
def generateFourierMatricesWithOmega(N):
    return createFourierMatrices(N, fourierGenerator)

In [33]:
IdentityAndDiagonal, FHalfNSize, EvenOddPermutation = generateFourierMatricesWithOmega(4)

In [34]:
assert sympy.Matrix([
    [1, 0, 1, 0],
    [0, 1, 0, ω],
    [1, 0, -1, 0],
    [0, 1, 0, -ω],
]) == IdentityAndDiagonal

IdentityAndDiagonal

Matrix([
[1, 0,  1,      0],
[0, 1,  0,  omega],
[1, 0, -1,      0],
[0, 1,  0, -omega]])

In [35]:
assert sympy.Matrix([
    [1, 1, 0, 0],
    [1, ω, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 1, ω]
]) == FHalfNSize

FHalfNSize

Matrix([
[1,     1, 0,     0],
[1, omega, 0,     0],
[0,     0, 1,     1],
[0,     0, 1, omega]])

In [36]:
FHalfNSize = sympy

In [37]:
assert sympy.Matrix([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
]) == EvenOddPermutation

EvenOddPermutation

Matrix([
[1, 0, 0, 0],
[0, 0, 1, 0],
[0, 1, 0, 0],
[0, 0, 0, 1]])

In [38]:
def generateFourierMatricesWithExp(N):
    return createFourierMatrices(N, fourierGeneratorWithExp)

In [39]:
IdentityAndDiagonal, FourierHalfNSize, EvenOddPermutation = generateFourierMatricesWithExp(4)

In [40]:
assert sympy.Matrix([
    [1, 0, 1, 0],
    [0, 1, 0, I],
    [1, 0, -1, 0],
    [0, 1, 0, -I]
]) == IdentityAndDiagonal

IdentityAndDiagonal

Matrix([
[1, 0,  1,  0],
[0, 1,  0,  I],
[1, 0, -1,  0],
[0, 1,  0, -I]])

In [41]:
assert sympy.Matrix([
    [1, 1, 0, 0],
    [1, -1, 0, 0],
    [0, 0, 1, 1],
    [0, 0, 1, -1]
]) == FourierHalfNSize

FourierHalfNSize

Matrix([
[1,  1, 0,  0],
[1, -1, 0,  0],
[0,  0, 1,  1],
[0,  0, 1, -1]])

In [42]:
assert sympy.Matrix([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
]) == EvenOddPermutation

EvenOddPermutation

Matrix([
[1, 0, 0, 0],
[0, 0, 1, 0],
[0, 1, 0, 0],
[0, 0, 0, 1]])

## Solution for \#6

In [53]:
IdentityAndDiagonal = sympy.Matrix([
    [1, 0, 0,  1,    0,    0],
    [0, 1, 0,  0,    ω,    0],
    [0, 0, 1,  0,    0,    ω**2],
    [1, 0, 0, -1,    0,    0],
    [0, 1, 0,  0,   -ω,    0],
    [0, 0, 1,  0,    0,   -ω**2],
])
IdentityAndDiagonal

Matrix([
[1, 0, 0,  1,      0,         0],
[0, 1, 0,  0,  omega,         0],
[0, 0, 1,  0,      0,  omega**2],
[1, 0, 0, -1,      0,         0],
[0, 1, 0,  0, -omega,         0],
[0, 0, 1,  0,      0, -omega**2]])

In [54]:
FourierHalfNSize = sympy.Matrix([
    [1, 1,    1,    0,    0,    0],
    [1, ω**2, ω**4, 0,    0,    0],
    [1, ω**4, ω**2, 0,    0,    0],
    [0, 0,    0,    1,    1,    1],
    [0, 0,    0,    1,    ω**2, ω**4],
    [0, 0,    0,    1,    ω**4, ω**2],
])

FourierHalfNSize

Matrix([
[1,        1,        1, 0,        0,        0],
[1, omega**2, omega**4, 0,        0,        0],
[1, omega**4, omega**2, 0,        0,        0],
[0,        0,        0, 1,        1,        1],
[0,        0,        0, 1, omega**2, omega**4],
[0,        0,        0, 1, omega**4, omega**2]])

In [55]:
EvenOddPermutation

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 1, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 1]])

In [56]:
FourierBasedOnMatrices = IdentityAndDiagonal * FourierHalfNSize * EvenOddPermutation
FourierBasedOnMatrices

Matrix([
[1,         1,        1,         1,        1,         1],
[1,     omega, omega**2,  omega**3, omega**4,  omega**5],
[1,  omega**2, omega**4,  omega**6, omega**2,  omega**4],
[1,        -1,        1,        -1,        1,        -1],
[1,    -omega, omega**2, -omega**3, omega**4, -omega**5],
[1, -omega**2, omega**4, -omega**6, omega**2, -omega**4]])

In [57]:
# assert FourierBasedOnMatrices == fourierGeneratorWithExp(N)

fourierGenerator(N)

Matrix([
[1,        1,         1,         1,         1,         1],
[1,    omega,  omega**2,  omega**3,  omega**4,  omega**5],
[1, omega**2,  omega**4,  omega**6,  omega**8, omega**10],
[1, omega**3,  omega**6,  omega**9, omega**12, omega**15],
[1, omega**4,  omega**8, omega**12, omega**16, omega**20],
[1, omega**5, omega**10, omega**15, omega**20, omega**25]])

In [58]:
N = 4
sympy.Matrix(
    sympy.fft(sympy.Matrix.eye(2, 2), 1)
)

Matrix([
[          2],
[1.0 - 1.0*I],
[          0],
[1.0 + 1.0*I]])

In [59]:
N = 16
half = int(N/2)

sympy.fft(sympy.Matrix.eye(2), 1)


[2, 1.0 - 1.0*I, 0, 1.0 + 1.0*I]

In [60]:
sympy.Matrix.eye(half)

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])